# Notebook Submission 1 Kelas MLOps: Analisis Sentimen Pada Masuknya Patrick Kluivert Sebagai Pelatih Timnas Indonesia

## Import Library

In [1]:
import tensorflow as tf
from tfx.components import Tuner, Trainer
import subprocess
import os
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2 
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tensorflow_transform as tft
from tfx.components.trainer.fn_args_utils import FnArgs

from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 

import tensorflow_model_analysis as tfma 
from tfx.components import Evaluator, Pusher 
from tfx.proto import trainer_pb2

from absl import logging
logging.set_verbosity(logging.INFO)

# Set Variable

In [2]:
# Variabel-variabel pipeline
PIPELINE_NAME = "sentiment-pipeline"
SCHEMA_PIPELINE_NAME = "sentiment-schema"

# Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path SQLite untuk MLMD Storage
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Direktori output untuk model
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# Direktori Data
PROCESSED_DATA_PATH = "data/youtube-comments/processed_comments.csv"
INGESTION_DATA_PATH = "data/youtube-comments"

# Direktori Root Modules
ROOT_MODULES = 'modules'

# Data Preprocessing
Menjalankan proses-proses data processing yang dilakukan pada kolom comment, serta membentuk *polarity*.

In [3]:
# Jalankan Preprocessing sebelum pipeline
print("🔄 Menjalankan preprocessing data...")

try:
    result = subprocess.run(
        ["python", f"{ROOT_MODULES}/sentiment_preprocess.py"], 
        check=True, 
        capture_output=True, 
        text=True
    )
    
    print("✅ Preprocessing berhasil dijalankan!")

except subprocess.CalledProcessError as e:
    print(f"❌ Error saat menjalankan preprocessing: {e}")
    print("📜 Log Error:")
    print(e.stderr)  
    raise

# Pastikan file hasil preprocessing tersedia sebelum melanjutkan pipeline
if not os.path.exists(PROCESSED_DATA_PATH):
    raise FileNotFoundError(f"❌ File hasil preprocessing tidak ditemukan: {PROCESSED_DATA_PATH}")

print(f"✅ File hasil preprocessing tersedia: {PROCESSED_DATA_PATH}")

🔄 Menjalankan preprocessing data...
✅ Preprocessing berhasil dijalankan!
✅ File hasil preprocessing tersedia: data/youtube-comments/processed_comments.csv


Menampilkan Pipeline yang sudah dibentuk

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Data Ingestion

Memisahkan data menjadi train dan evaluation

In [5]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

example_gen = CsvExampleGen(input_base=INGESTION_DATA_PATH, output_config=output)

In [6]:
interactive_context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data data/youtube-comments\* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Data Validation

## Summary Statistics

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to pipelines\sentiment-pipeline\StatisticsGen\statistics\2\Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to pipelines\sentiment-pipeline\StatisticsGen\statistics\2\Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Menampilkan informasi statistik dasar dari dataset yang digunakan. Statistik yang ditampilkan sudah dalam bentuk splitted data *train* dan *eval*.

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

# Make Data Schema

In [9]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])
interactive_context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to pipelines\sentiment-pipeline\SchemaGen\schema\3\schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Menampilkan hasil skema data, dengan menunjukkan fitur-fitur yang ada.

In [10]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'comment',BYTES,required,,-
'polarity',INT,required,,-


# Identify Each Anomalies

In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to pipelines\sentiment-pipeline\ExampleValidator\anomalies\4\Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to pipelines\sentiment-pipeline\ExampleValidator\anomalies\4\Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [12]:
interactive_context.show(example_validator.outputs['anomalies'])

# Component Transform

In [13]:
TRANSFORM_MODULE_FILE = 'modules/sentiment_transform.py'

In [14]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

INFO:absl:Generating ephemeral wheel package for 'd:\\belajar data\\Dicoding\\MLOps\\Submission\\faishal_ali-pipeline\\modules\\sentiment_transform.py' (including modules: ['sentiment_preprocess', 'sentiment_trainer', 'sentiment_transform', 'sentiment_tuner']).
INFO:absl:User module package has hash fingerprint version dc7c7589150e3aea558171b58f7b2938262550af183f2574b16bf763a6863b31.
INFO:absl:Executing: ['c:\\Users\\legion\\miniconda3\\envs\\mlops\\python.exe', 'C:\\Users\\legion\\AppData\\Local\\Temp\\tmp3lj3s466\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\legion\\AppData\\Local\\Temp\\tmpba7un8ax', '--dist-dir', 'C:\\Users\\legion\\AppData\\Local\\Temp\\tmpdtf5f2l_']
INFO:absl:Successfully built user code wheel distribution at 'pipelines\\sentiment-pipeline\\_wheels\\tfx_user_code_Transform-0.0+dc7c7589150e3aea558171b58f7b2938262550af183f2574b16bf763a6863b31-py3-none-any.whl'; target user module is 'sentiment_transform'.
INFO:absl:Full user module path is 's

INFO:tensorflow:Assets written to: pipelines\sentiment-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\4d4939a2e8824cd8bde87523a1fc76fe\assets


INFO:tensorflow:Assets written to: pipelines\sentiment-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\4d4939a2e8824cd8bde87523a1fc76fe\assets
INFO:absl:Writing fingerprint to pipelines\sentiment-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\4d4939a2e8824cd8bde87523a1fc76fe\fingerprint.pb


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Modelling

## Make A Trainer

In [15]:
TUNER_MODULE_FILE = "modules/sentiment_tuner.py"
TRAINER_MODULE_FILE = "modules/sentiment_trainer.py"

In [22]:
tuner = Tuner(
    module_file=TUNER_MODULE_FILE,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    train_args=trainer_pb2.TrainArgs(num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(num_steps=1000),
)

interactive_context.run(tuner)

INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'embedding_dim', 'default': 32, 'conditions': [], 'values': [32, 128], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'lstm_units', 'default': 64, 'conditions': [], 'values': [64, 128], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'dropout_rate', 'default': 0.3, 'conditions': [], 'values': [0.3, 0.4], 'ordered': True}}], 'values': {'embedding_dim': 32, 'lstm_units': 128, 'dropout_rate': 0.3, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}}
INFO:absl:Best Hyperparameters are written to pipelines\sentiment-pipeline\Tuner\best_hyperparameters\12\best_hyperparameters.txt.


Trial 8 Complete [00h 08m 27s]
val_accuracy: 0.7916666865348816

Best val_accuracy So Far: 0.7958333492279053
Total elapsed time: 00h 26m 28s
Results summary
Results in pipelines\sentiment-pipeline\.temp\12\sentiment_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0003 summary
Hyperparameters:
embedding_dim: 32
lstm_units: 128
dropout_rate: 0.3
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.7958333492279053

Trial 0006 summary
Hyperparameters:
embedding_dim: 32
lstm_units: 64
dropout_rate: 0.4
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.7947916388511658

Trial 0004 summary
Hyperparameters:
embedding_dim: 32
lstm_units: 64
dropout_rate: 0.3
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.793749988079071

Trial 0000 summary
Hyperparameters:
embedding_dim: 128
lstm_units: 64
dropout_rate: 0.3
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner

TypeError: '<' not supported between instances of 'int' and 'NoneType'

In [23]:
trainer  = Trainer(
    module_file=TRAINER_MODULE_FILE,
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

INFO:absl:Generating ephemeral wheel package for 'd:\\belajar data\\Dicoding\\MLOps\\Submission\\faishal_ali-pipeline\\modules\\sentiment_trainer.py' (including modules: ['sentiment_preprocess', 'sentiment_trainer', 'sentiment_transform', 'sentiment_tuner']).
INFO:absl:User module package has hash fingerprint version 68ba40788f69cf8e76b5ff94a97a002f57ae603686602c82c73c88d8fa8bb862.
INFO:absl:Executing: ['c:\\Users\\legion\\miniconda3\\envs\\mlops\\python.exe', 'C:\\Users\\legion\\AppData\\Local\\Temp\\tmpn01hpkh4\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\legion\\AppData\\Local\\Temp\\tmpelbv0dyk', '--dist-dir', 'C:\\Users\\legion\\AppData\\Local\\Temp\\tmp6ap6iffs']
INFO:absl:Successfully built user code wheel distribution at 'pipelines\\sentiment-pipeline\\_wheels\\tfx_user_code_Trainer-0.0+68ba40788f69cf8e76b5ff94a97a002f57ae603686602c82c73c88d8fa8bb862-py3-none-any.whl'; target user module is 'sentiment_trainer'.
INFO:absl:Full user module path is 'sentime

Total training samples: 4008, Steps per epoch: 62
Total validation samples: 992, Validation steps: 15
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 comment_xf (InputLayer)     [(None, 200)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 200, 128)          489216    
                                                                 
 bidirectional_1 (Bidirecti  (None, 256)               263168    
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)       

KeyboardInterrupt: 